In [2]:
# import the necessary modules
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import fpgrowth, association_rules

In [3]:
basket_items = pd.read_csv("groceries_basket.csv")
basket_items.head()

,order_id,product_id,product_name,category,add_to_cart_sequence_index
0,1,49302,Bulgarian Yogurt,dairy eggs,1
1,1,10246,Organic Celery Hearts,produce,2
2,1,43633,Lightly Smoked Sardines in Olive Oil,canned goods,3
3,1,13176,Bag of Organic Bananas,produce,4
4,1,47209,Organic Hass Avocado,produce,5


In [4]:
# items
n_items = basket_items['product_id'].nunique()
print(f'Number of unique items: {n_items}')
# orders
n_orders = basket_items['order_id'].nunique()
print(f'Number of unique orders: {n_orders}')
# basket_items
n_basket_items = len(basket_items)
print(f'Number of basket items: {n_basket_items}')

Number of unique items: 26548
Number of unique orders: 64864
Number of basket items: 573124


# a) Product Frequencies

In [ ]:
# Support count per product = number of distinct orders containing the product
product_support = (
    basket_items.groupby("product_id")["order_id"]
    .nunique()
    .rename("support_count")
    .sort_values(ascending=False)
)

if "n_orders" not in globals():
    n_orders = basket_items["order_id"].nunique()

support_threshold_1pct = int(np.ceil(0.01 * n_orders))
mean_support_count = product_support.mean()
median_support_count = product_support.median()

print("1) Minimum support count for >= 1% support:", support_threshold_1pct)
print("2) Mean support count across all products:", mean_support_count)
print("3) Median support count across all products:", median_support_count)

# Helper mapping for later tasks (product_id -> product_name)
product_name_by_id = (
    basket_items.drop_duplicates("product_id").set_index("product_id")["product_name"]
)


# b) + c) Most frequent Products

You can separate these if you want, but it makes sense to do them together.

In [ ]:
from IPython.display import display

# Absolute + relative frequencies for the 10 most frequent products
top10 = product_support.head(10).to_frame()
top10["support"] = top10["support_count"] / n_orders
top10["product_name"] = top10.index.map(product_name_by_id)
top10 = top10[["product_name", "support_count", "support"]]

display(top10)

top10_ids = top10.index.tolist()

# Max possible support for a 3-itemset using this list
# (upper bound = min support among the chosen 3 items)
top3_ids = top10.index[:3].tolist()
max_possible_support_count = int(top10.loc[top3_ids, "support_count"].min())
max_possible_support = max_possible_support_count / n_orders

print("\nMax possible support count for a 3-itemset (upper bound):", max_possible_support_count)
print("Max possible support (relative):", max_possible_support)
print("Itemset product IDs:", top3_ids)
print("Itemset product names:", [product_name_by_id.loc[i] for i in top3_ids])

# c) How many orders actually contain all three products?
orders_with_all_three = (
    basket_items[basket_items["product_id"].isin(top3_ids)]
    .groupby("order_id")["product_id"]
    .nunique()
    .eq(3)
)
actual_intersection_count = int(orders_with_all_three.sum())

print("\nOrders containing all three products:", actual_intersection_count)
print("Support of the 3-itemset (relative):", actual_intersection_count / n_orders)

print(
    "\nExplanation: 'Banana' and 'Bag of Organic Bananas' are substitutes, so customers usually buy one of them "
    "instead of both; therefore the triple intersection with 'Organic Strawberries' is very rare."
)


# d) + e) Product Categories

You don't really need the notebook for e)

In [ ]:
# d) Total number of sold products (basket items) per category
items_sold_per_category = (
    basket_items.groupby("category").size().rename("items_sold").sort_index()
)

ax = items_sold_per_category.plot(kind="bar", figsize=(12, 4))
ax.set_title("Total number of sold products (basket items) per category")
ax.set_xlabel("category")
ax.set_ylabel("items_sold")
plt.xticks(rotation=45, ha="right")
plt.tight_layout()
plt.show()

# d) Mean + median number of items per order for each product category
items_per_order_per_category = (
    basket_items.groupby(["category", "order_id"]).size().rename("items_per_order")
)
category_order_stats = (
    items_per_order_per_category.groupby("category").agg(["mean", "median"]).sort_index()
)

ax = category_order_stats.plot(kind="bar", figsize=(12, 4))
ax.set_title("Mean and median items per order per category")
ax.set_xlabel("category")
ax.set_ylabel("items_per_order")
plt.xticks(rotation=45, ha="right")
plt.tight_layout()
plt.show()

# e) What mean vs median tell you about beverages
bev_mean = category_order_stats.loc["beverages", "mean"]
bev_median = category_order_stats.loc["beverages", "median"]

print(f"Beverages - mean items/order: {bev_mean:.3f}, median items/order: {bev_median:.1f}")
print(
    "Interpretation: The median of 1 shows that most beverage-containing orders include exactly one beverage item. "
    "The higher mean indicates that a smaller number of orders contain multiple beverage items, which pulls the average up."
)


# f) Frequent Itemsets

In [5]:
import re
from collections import Counter

# use the provided frequent itemsets
frequent_itemsets = pd.read_csv("frequent_itemsets.csv")

# Parse strings like "frozenset({47209, 24852})" into Python frozensets of ints
frequent_itemsets["itemset_ids"] = frequent_itemsets["itemsets"].astype(str).apply(
    lambda s: frozenset(int(x) for x in re.findall(r"\d+", s))
)
frequent_itemsets["size"] = frequent_itemsets["itemset_ids"].apply(len)

# Overwrite the original string column with real frozensets so mlxtend can use it
frequent_itemsets["itemsets"] = frequent_itemsets["itemset_ids"]

# Ensure we have a product_id -> name mapping (defined in a), even if cells were run out of order
if "product_name_by_id" not in globals():
    product_name_by_id = (
        basket_items.drop_duplicates("product_id")
        .set_index("product_id")["product_name"]
    )

# 1) Names of the products in the largest frequent itemset
max_size = frequent_itemsets["size"].max()
largest_itemsets = frequent_itemsets.loc[
    frequent_itemsets["size"].eq(max_size), "itemset_ids"
].tolist()

print("1) Largest frequent itemset size:", int(max_size))
print("   Number of largest itemsets:", len(largest_itemsets))
for i, itemset_ids in enumerate(largest_itemsets, start=1):
    ids_sorted = sorted(itemset_ids)
    print(f"   Itemset {i} (IDs):", ids_sorted)
    print("   Names:")
    for pid in ids_sorted:
        print("   -", product_name_by_id.get(pid, f"<unknown {pid}>"))

# 2) How many frequent itemsets of size 2 or larger (absolute and relative)?
n_ge2 = int((frequent_itemsets["size"] >= 2).sum())
share_ge2 = n_ge2 / len(frequent_itemsets)
print("\n2) Frequent itemsets with size >= 2:", n_ge2)
print("   Relative:", share_ge2)

# 3) Which item appears in the most frequent itemsets?
item_counts = Counter()
for s in frequent_itemsets["itemset_ids"]:
    item_counts.update(s)

most_common_item_id, most_common_item_count = item_counts.most_common(1)[0]
print("\n3) Item in the most frequent itemsets:", most_common_item_id)
print("   Name:", product_name_by_id.get(most_common_item_id, f"<unknown {most_common_item_id}>"))
print("   Appears in:", most_common_item_count)

# 4) How many products appear in only a single frequent itemset?
n_only_once = sum(1 for v in item_counts.values() if v == 1)
print("\n4) Products appearing in only one frequent itemset:", n_only_once)

frequent_itemsets.head()

,support,itemsets
0,0.118725,(13176)
1,0.055578,(47209)
2,0.015432,(22035)
3,0.008048,(10246)
4,0.029462,(46979)


# g) + h) Category Sets

### h) Why the observations are not unexpected

1. Produce accounts for the largest share of basket items and most of the top-frequency products are produce. Therefore, produce-produce combinations are most likely to reach the minimum support and become frequent itemsets.
2. Since produce items are present in a large fraction of orders, including at least one produce item greatly increases the chance of surpassing the support threshold. Hence, nearly all frequent itemsets include produce and only few itemsets without produce remain.
3. Although many orders contain no beverages, the customers who do buy beverages often buy the same beverage combinations repeatedly, so some beverage-only itemsets still reach the minimum support. The fact that only a few beverage-only itemsets are frequent indicates that frequent beverage sales are concentrated on a small subset of beverages (limited diversity among the frequent beverages).

In [ ]:
from IPython.display import display

# g) Consider frequent itemsets with size >= 2 and convert them to category sets
product_category_by_id = (
    basket_items.drop_duplicates("product_id").set_index("product_id")["category"]
)

fi_ge2 = frequent_itemsets[frequent_itemsets["itemsets"].apply(len) >= 2].copy()
fi_ge2["category_set"] = fi_ge2["itemsets"].apply(
    lambda s: frozenset(product_category_by_id.get(pid, "UNKNOWN") for pid in s)
)

category_set_counts = fi_ge2["category_set"].value_counts()
category_sets = category_set_counts.rename_axis("category_set").reset_index(name="frequency")
category_sets["label"] = category_sets["category_set"].apply(
    lambda s: "{" + ", ".join(sorted(s)) + "}"
)

display(category_sets)

# Plot: category sets ordered by frequency (descending)
ax = category_sets.set_index("label")["frequency"].plot(kind="bar", figsize=(12, 4))
ax.set_title("Category-set frequencies for frequent itemsets (size >= 2)")
ax.set_xlabel("category set")
ax.set_ylabel("frequency (# frequent itemsets)")
plt.xticks(rotation=45, ha="right")
plt.tight_layout()
plt.show()

# Helpful numbers for h)
n_itemsets_ge2 = len(fi_ge2)
n_produce_only = int(category_set_counts.get(frozenset({"produce"}), 0))
n_without_produce = int(fi_ge2["category_set"].apply(lambda s: "produce" not in s).sum())
n_beverages_only = int(category_set_counts.get(frozenset({"beverages"}), 0))

print("Frequent itemsets with size >= 2:", n_itemsets_ge2)
print("- only produce:", n_produce_only)
print("- without produce:", n_without_produce)
print("- only beverages:", n_beverages_only)


# i) Association Rules and Frequent Items

You don't really need the notebook for j)

In [6]:
# i) Determine association rules with support count >= 100 and confidence >= 0.2
if "n_orders" not in globals():
    n_orders = basket_items["order_id"].nunique()

min_support = 100 / n_orders

# mlxtend expects a DataFrame with columns: support, itemsets
rules = association_rules(
    frequent_itemsets[["support", "itemsets"]],
    metric="confidence",
    min_threshold=0.2,
)

# Ensure the minimum support count of 100 (given the mined itemsets, this should already hold)
rules = rules[rules["support"] >= min_support].copy()

# Add absolute support count for convenience
rules["support_count"] = (rules["support"] * n_orders).round().astype(int)

# Drop extra metrics (depending on mlxtend version)
rules = rules.drop(
    columns=[
        "representativity",
        "leverage",
        "conviction",
        "zhangs_metric",
        "jaccard",
        "certainty",
        "kulczynski",
    ],
    errors="ignore",
)

rules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift
0,(47209),(13176),0.055578,0.118725,0.018423,0.331484,2.792025
1,(47209),(21137),0.055578,0.084484,0.011486,0.206657,2.446100
2,"(13176, 47209)",(21137),0.018423,0.084484,0.005396,0.292887,3.466756
3,"(13176, 21137)",(47209),0.023942,0.055578,0.005396,0.225370,4.055039
4,"(47209, 21137)",(13176),0.011486,0.118725,0.005396,0.469799,3.957021


In [ ]:
# i) Answers

# 1) Mean and median lift across all rules
mean_lift = rules["lift"].mean()
median_lift = rules["lift"].median()
print("1) Mean lift:", mean_lift)
print("   Median lift:", median_lift)

# 2) / 3) Count rules that contain at least one of the top-10 products
if "top10_ids" not in globals():
    product_support_tmp = (
        basket_items.groupby("product_id")["order_id"].nunique().sort_values(ascending=False)
    )
    top10_ids = product_support_tmp.head(10).index.tolist()

top10_set = set(top10_ids)

n_top10_in_antecedent = int(
    rules["antecedents"].apply(lambda s: len(set(s) & top10_set) > 0).sum()
)
n_top10_in_consequent = int(
    rules["consequents"].apply(lambda s: len(set(s) & top10_set) > 0).sum()
)

print("\n2) #rules with >= 1 top-10 product in the antecedent:", n_top10_in_antecedent)
print("3) #rules with >= 1 top-10 product in the consequent:", n_top10_in_consequent)


# j) Confidence

In [ ]:
from IPython.display import display

# j) Rule with the highest confidence
best_rule = rules.sort_values(["confidence", "support"], ascending=[False, False]).iloc[0]

def itemset_to_names(itemset):
    return [product_name_by_id.get(pid, str(pid)) for pid in sorted(itemset)]

ant_names = itemset_to_names(best_rule["antecedents"])
con_names = itemset_to_names(best_rule["consequents"])

display(best_rule.to_frame().T)

print("Rule:")
print("A =", ant_names)
print("B =", con_names)
print(
    f"support_count={best_rule['support_count']}, confidence={best_rule['confidence']:.3f}, lift={best_rule['lift']:.3f}"
)

print(
    "\nInterpretation: Customers who buy the antecedent products also buy the consequent products with the shown confidence. "
    "Since the lift is > 1, the consequent becomes more likely than under independence."
)

print(
    "\nWhy frequent items often appear as consequents: Confidence conditions on the antecedent (support(A) is the denominator). "
    "If a very frequent product were the antecedent, achieving a very high confidence would require the consequent to appear in a large fraction of all those orders. "
    "In contrast, predicting a frequent product as the consequent is easier because it already has a high base rate."
)


# k) No very frequent products, Lift

In [ ]:
from IPython.display import display

# k) Rules without any of the top-10 most frequent products in antecedent or consequent
if "top10_ids" not in globals():
    product_support_tmp = (
        basket_items.groupby("product_id")["order_id"].nunique().sort_values(ascending=False)
    )
    top10_ids = product_support_tmp.head(10).index.tolist()

top10_set = set(top10_ids)

mask_no_top10 = (
    rules["antecedents"].apply(lambda s: len(set(s) & top10_set) == 0)
    & rules["consequents"].apply(lambda s: len(set(s) & top10_set) == 0)
)
rules_no_top10 = rules[mask_no_top10].copy()

print("Overall rules:", len(rules))
print("- mean lift:", rules["lift"].mean())
print("- median lift:", rules["lift"].median())

print("\nRules without top-10 products:", len(rules_no_top10))
print("- mean lift:", rules_no_top10["lift"].mean())
print("- median lift:", rules_no_top10["lift"].median())

display(
    rules_no_top10.sort_values("lift", ascending=False)
    .head(10)[["antecedents", "consequents", "support_count", "confidence", "lift"]]
)

print(
    "\nStriking observation: These rules typically have a much higher lift than the overall set of rules. "
    "This is not surprising because lift compares observed co-occurrence to what we would expect under independence, "
    "and for less frequent items the expected co-occurrence is very small—so strong co-purchases lead to large lift values."
)


# l) Sub-sequences

In [34]:
# function from the exercise notebook
def contained(is_seq1, is_seq2):
    size1 = len(is_seq1)
    size2 = len(is_seq2)

    if size1 > size2:
        return False

    last_1 = 0
    last_2 = 0

    while last_1 < size1 and last_2 < size2:
        item1 = is_seq1[last_1]
        item2 = is_seq2[last_2]
        if item1 == item2:
            last_1 += 1
        last_2 += 1

    if last_1 == size1:
        return True
    else:
        return False

In [ ]:
from IPython.display import display

# l) Subsequence support counts for bi-directional singleton rules
# The assignment calls this column add_to_cart_order; in this dataset it is add_to_cart_sequence_index
sequence_col = (
    "add_to_cart_order"
    if "add_to_cart_order" in basket_items.columns
    else "add_to_cart_sequence_index"
)

# Only consider rules with single-item antecedent and consequent
rules_single = rules[
    (rules["antecedents"].apply(len) == 1) & (rules["consequents"].apply(len) == 1)
].copy()

# Extract directed pairs (a -> b)
directed_pairs = [
    (next(iter(a)), next(iter(b)))
    for a, b in zip(rules_single["antecedents"], rules_single["consequents"])
]
directed_set = set(directed_pairs)

# Keep pairs where both directions exist (a -> b and b -> a)
bidirectional_pairs = sorted(
    {tuple(sorted((a, b))) for (a, b) in directed_set if (b, a) in directed_set}
)

print("Number of bi-directional singleton pairs:", len(bidirectional_pairs))
bidirectional_pairs

# Compute support counts for subsequences <a,b> and <b,a>
unique_products = sorted({pid for pair in bidirectional_pairs for pid in pair})

subset = basket_items[basket_items["product_id"].isin(unique_products)][
    ["order_id", "product_id", sequence_col]
]

# Position of each product in each order (min index; products occur at most once in this dataset)
positions = subset.groupby(["order_id", "product_id"])[sequence_col].min().unstack("product_id")

rows = []
for a, b in bidirectional_pairs:
    pa = positions[a]
    pb = positions[b]
    both = pa.notna() & pb.notna()

    support_ab = int((both & (pa < pb)).sum())
    support_ba = int((both & (pb < pa)).sum())

    rows.append(
        {
            "a_id": a,
            "a_name": product_name_by_id.get(a, str(a)),
            "b_id": b,
            "b_name": product_name_by_id.get(b, str(b)),
            "support_<a,b>": support_ab,
            "support_<b,a>": support_ba,
            "orders_with_both": int(both.sum()),
        }
    )

subsequence_support = pd.DataFrame(rows).sort_values("orders_with_both", ascending=False)
display(subsequence_support)


# m) Rules with support count > 99, confidence > 0.2, Lift < 1

You don't need any new calculations for this. Rather consult the lecture slides and your answer for b)